## Z3 Theorem Prover

In [ ]:
from z3 import *
import re

def clean_z3_code(code_block):
    """
    Clean the Z3 code block by removing unnecessary syntax and comments.
    """
    # Remove 'smt2' if present
    code_block = re.sub(r'^smt2\s*', '', code_block)
    
    # Remove any trailing whitespace
    lines = [line.strip() for line in code_block.split('\n')]
    
    # Filter out empty lines and join back
    return '\n'.join(line for line in lines if line)

def execute_z3_code(code_block):
    """
    Execute a Z3 code block and return the solver results.
    """
    # Clean the code block first
    code_block = clean_z3_code(code_block)
    
    # Create a complete namespace for Z3 execution
    namespace = {
        # Sorts
        'StringSort': StringSort,
        'IntSort': IntSort,
        'IntegerSort': IntSort,
        'BoolSort': BoolSort,
        'RealSort': RealSort,
        'ArraySort': ArraySort,
        'SeqSort': SeqSort,
        'SequenceSort': SeqSort,
        
        # Functions and operations
        'Function': Function,
        'StringVal': StringVal,
        'String': StringVal,
        'IntVal': IntVal,
        'Int': IntVal,
        'RealVal': RealVal,
        'Real': RealVal,
        'BoolVal': BoolVal,
        'Bool': BoolVal,
        'Not': Not,
        'And': And,
        'Or': Or,
        'Implies': Implies,
        'If': If,
        'eq': eq,
        'Equal': eq,  # Alias for Equal to avoid errors
        'Eq': eq,
        
        # Additional Z3 utilities
        'Solver': Solver,
        'sat': sat,
        'unsat': unsat,
        'unknown': unknown,

    }
    
    try:
        # Execute the code block to get predicates, constants, axioms, and query
        exec(code_block, namespace)
        
        # Get axioms and query from the namespace
        axioms = namespace.get('axioms', [])
        query = namespace.get('query')
        
        if not axioms or query is None:
            return None, None, "Invalid code block: missing axioms or query"
        
        # Create solver and add axioms
        solver = Solver()
        for axiom in axioms:
            solver.add(axiom)
        
        # Add negation of query
        solver.add(Not(query))
        
        # Check satisfiability
        result = solver.check()
        
        if result == unsat:
            return axioms, query, "Query is true"
        elif result == sat:
            model = solver.model()
            return axioms, query, f"Query is false: {model}"
        else:
            return axioms, query, "Query is unknown"
        
        
    except Exception as e:
        return None, None, f"Error executing code: {str(e)}"




In [ ]:
def extract_code_blocks(text):
    """
    Extract right_answer, right_answer_z3, hallucinated_answer, and hallucinated_answer_z3 
    code blocks from the formatted text.
    
    Args:
        text (str): The full text content
        
    Returns:
        list: List of tuples containing 
              (iteration, right_answer, right_answer_z3, hallucinated_answer, hallucinated_answer_z3)
    """
    iterations = []
    current_block = {}
    current_type = None
    iteration = -1
    
    for line in text.split('\n'):
        if line.startswith('Iteration'):
            if current_block:
                iterations.append((
                    iteration, 
                    current_block.get('knowledge_question', ''), 
                    current_block.get('right_answer', ''), 
                    current_block.get('right_answer_z3', ''), 
                    current_block.get('hallucinated_answer', ''), 
                    current_block.get('hallucinated_answer_z3', '')
                ))
            iteration = int(line.split()[1].strip(':'))
            current_block = {}
        elif line.startswith('knowledge_question'):
            current_type = 'knowledge_question'
            current_block[current_type] = ''
        elif line.startswith('right_answer:'):
            current_type = 'right_answer'
            current_block[current_type] = ''
        elif line.startswith('right_answer_z3:'):
            current_type = 'right_answer_z3'
            current_block[current_type] = ''
        elif line.startswith('hallucinated_answer:'):
            current_type = 'hallucinated_answer'
            current_block[current_type] = ''
        elif line.startswith('hallucinated_answer_z3:'):
            current_type = 'hallucinated_answer_z3'
            current_block[current_type] = ''
        elif line.startswith('-' * 40):
            current_type = None
        elif current_type and not line.startswith('=='):
            current_block[current_type] = current_block[current_type] + line + '\n'
    
    # Add the last block
    if current_block:
        iterations.append((
            iteration,
            current_block.get('knowledge_question', ''), 
            current_block.get('right_answer', ''), 
            current_block.get('right_answer_z3', ''), 
            current_block.get('hallucinated_answer', ''), 
            current_block.get('hallucinated_answer_z3', '')
        ))
    
    return iterations


def test_z3_code(iteration, knowledge_question, right_answer, right_answer_z3, hallucinated_answer, hallucinated_answer_z3, COUNT_1, COUNT_2):
    """
    Test both right and hallucinated Z3 answers and print results.
    """
    print(f"\nProcessing Iteration {iteration}")
    print("=" * 50)
    
    print("\nTesting Right Answer Z3:")
    print("-" * 30)
    try:
        axioms, query, result_1 = execute_z3_code(right_answer_z3)
        if axioms is not None and query is not None:
            print("Axioms:")
            for axiom in axioms:
                print(f"  {axiom}")
            print(f"Query: {query}")
        print(f"Result: {result_1}")
        
        if result_1 == "Query is true":
            COUNT_1 += 1
        
    except Exception as e:
        print(f"Error: {str(e)}")
    
    
    print("\nTesting Hallucinated Answer Z3:")
    print("-" * 30)
    try:
        axioms, query, result_2 = execute_z3_code(hallucinated_answer_z3)
        if axioms is not None and query is not None:
            print("Axioms:")
            for axiom in axioms:
                print(f"  {axiom}")
            print(f"Query: {query}")
        print(f"Result: {result_2}")
        
        if "Query is false" in result_2:
            COUNT_2 += 1
        
#         if "Error" in result_2:
#             tmp = knowledge_question[:-1] + "error\": " + "\"" + result_2 + "\", " + "\"right_response\": " + right_answer[:-1] + ", " +"\"hallucinated_response\": " + hallucinated_answer[:-1] + "}" + "\n"
#             with open("halu_dialogue_syntax_error.txt", 'a') as f:
#                 f.write(tmp)
# #             print("\n")
# #             COUNT += 1
            
    except Exception as e:
        print(f"Error: {str(e)}")
    
    return COUNT_1, COUNT_2


# Read the formatted snippets file
with open('formatted_snippets_dialogue.txt', 'r') as f:
    content = f.read()
    print(content)

# Extract code blocks
iterations = extract_code_blocks(content)



# # Process each iteration
COUNT_1 = 0
COUNT_2 = 0
TOTAL = 0
for iteration, knowledge_question, right_answer, right_answer_z3, hallucinated_answer, hallucinated_answer_z3 in iterations:
#     print(knowledge_question)
    COUNT_1, COUNT_2 = test_z3_code(iteration, knowledge_question, right_answer, right_answer_z3, hallucinated_answer, hallucinated_answer_z3, COUNT_1, COUNT_2)
    TOTAL += 1
    
print(TOTAL, COUNT_1)
print(TOTAL, COUNT_2)

print("Correct Answer Detection Accuracy: ", str((COUNT_1/TOTAL)*100), "%")
print("Hallucinated Answer Detection Accuracy: ", str((COUNT_2/TOTAL)*100), "%")